# Movie Rating Prediction with Python
### Goal: Analyze historical movie data and develop a model that predicts the rating of a moive based on features like genre, director, and actors. 

In [3]:
# imports
import pandas as pd   
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns 

Data Preprocessing

In [13]:
# load data
df = pd.read_csv("./IMDb.csv", na_values="?",encoding='latin-1')

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15509 entries, 0 to 15508
Data columns (total 10 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Name      15509 non-null  object 
 1   Year      14981 non-null  object 
 2   Duration  7240 non-null   object 
 3   Genre     13632 non-null  object 
 4   Rating    7919 non-null   float64
 5   Votes     7920 non-null   object 
 6   Director  14984 non-null  object 
 7   Actor 1   13892 non-null  object 
 8   Actor 2   13125 non-null  object 
 9   Actor 3   12365 non-null  object 
dtypes: float64(1), object(9)
memory usage: 1.2+ MB


In [15]:
df.describe()

,Rating
count,7919.000000
mean,5.841621
std,1.381777
min,1.100000
25%,4.900000
50%,6.000000
75%,6.800000
max,10.000000
